In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
# parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = 'logs/single_layer'

Use a single-layer perceptron as example: 

$pred = softmax(W x+b)$

In [3]:
# graph input
x = tf.placeholder('float', [None, 784], name='data')
y = tf.placeholder('float', [None, 10], name='label')

# model variables: W, b
W = tf.Variable(tf.zeros([784,10]), name='weights')
b = tf.Variable(tf.zeros([10]), name='bias')

In [4]:
# construct model

'put the model ops into SCOPEs, making tensorboard more convinent'
with tf.name_scope('Model'):
    logits = tf.matmul(x,W)+b
    pred = tf.nn.softmax( logits )
with tf.name_scope('Loss'):
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, y) )
#     cost = tf.reduce_mean( -tf.reduce_sum(y*tf.log(pred), reduction_indices=1) ) 
with tf.name_scope('sgd'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('evaluation'):
    corr_pred = tf.equal( tf.argmax(pred,1), tf.argmax(y,1) )
    acc = tf.reduce_mean(tf.cast(corr_pred, 'float'))

init = tf.initialize_all_variables()

In [5]:
'''create *summary* ops to monitor the cost/accuracy'''

loss_summary = tf.scalar_summary(tags='loss', values=cost)
accu_summary = tf.scalar_summary('accuracy', acc)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries() 
'''
using `tf.merge_all_summaries()` might cause invalid argument error when run for several times, see:
http://stackoverflow.com/questions/35413618/tensorflow-placeholder-error-when-using-tf-merge-all-summaries
'''

merged_summary_op = tf.merge_summary([loss_summary, accu_summary])

In [6]:
# launch the graph

with tf.Session() as sess:
    sess.run(init)
    
    '===write logs to tensorboard, using a summary writer==='
    summary_writer = tf.train.SummaryWriter(logdir=logs_path, graph=tf.get_default_graph())
    
    for epoch in xrange(1, training_epochs+1):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in xrange(total_batch):
            xs, ys = mnist.train.next_batch(batch_size)
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                    feed_dict = {x:xs, y:ys})
            '===write summary in each iteration==='
            summary_writer.add_summary(summary, epoch*total_batch+i)
            avg_cost += c/total_batch
        if epoch % display_step == 0:
            print 'epoch %4d, cost = %.9f' % (epoch, avg_cost)
    print 'finish!'
    print 'test accuracy: %.9f' % acc.eval({x:mnist.test.images, y:mnist.test.labels})
            

epoch    1, cost = 1.182138980
epoch    2, cost = 0.664671981
epoch    3, cost = 0.552553307
epoch    4, cost = 0.498565498
epoch    5, cost = 0.465381316
epoch    6, cost = 0.442502840
epoch    7, cost = 0.425479926
epoch    8, cost = 0.412139379
epoch    9, cost = 0.401361033
epoch   10, cost = 0.392376127
epoch   11, cost = 0.384698953
epoch   12, cost = 0.378102806
epoch   13, cost = 0.372392310
epoch   14, cost = 0.367289127
epoch   15, cost = 0.362701092
epoch   16, cost = 0.358608231
epoch   17, cost = 0.354868109
epoch   18, cost = 0.351416365
epoch   19, cost = 0.348292059
epoch   20, cost = 0.345420437
epoch   21, cost = 0.342687452
epoch   22, cost = 0.340259324
epoch   23, cost = 0.337911762
epoch   24, cost = 0.335755219
epoch   25, cost = 0.333633333
finish!
test accuracy: 0.913800001


Run `tensorboard --logdir=logs/` to open tensorboard, where we can see evolution of the accuracy/loss, and have a visulization of the graph structure. 